# Goedaardig vs kwaadaardige tumoren - Support Vector Machines

We voorspellen of een tumor goedaardig (_benign_ of kortweg _B_) is, of de patiënt weldegelijk borstkanker heeft en de tumor dus kwaardaardig (_malignant_ of kortweg _M_) is.

In [45]:
from toepassing_procedures import *
import numpy as np
# data inladen
import pandas as pd
from pandas import read_csv
# data opsplitsen in trainings-, validatie- en testset
from sklearn.model_selection import train_test_split
# grafieken
from matplotlib import pyplot as plt
# nodig voor het maken van het svm model
from sklearn import svm
# nodig om accuraatheid van de voorspellingen te bekijken
from sklearn import metrics

In [46]:
# We laden de data
gegevens = read_csv("data.csv")

# We genereren een random random_state
random_state = np.random.randint(100)

# We verdelen de data in 50% training, 25% validatie en 25% test
tmp, testData = train_test_split(gegevens, test_size=0.25, random_state=random_state)
trainingData, validatieData = train_test_split(tmp, test_size=0.33, random_state=random_state)

# Print the shapes of the resulting datasets
print(f"Grootte training data: \t{trainingData.shape} (50%)")
print(f"Grootte validatie data: {validatieData.shape} (25%)")
print(f"Grootte test data: \t{testData.shape} (25%)")


Grootte training data: 	(285, 33) (50%)
Grootte validatie data: (141, 33) (25%)
Grootte test data: 	(143, 33) (25%)


In [47]:
print(trainingData.head())

           id diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
317    894326         M       18.220         18.87          118.70     1027.0   
508    915452         B       16.300         15.70          104.70      819.8   
46   85713702         B        8.196         16.84           51.71      201.9   
42     855625         M       19.070         24.81          128.30     1104.0   
180    873592         M       27.220         21.87          182.10     2250.0   

     smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
317          0.09746           0.11170         0.11300             0.079500   
508          0.09427           0.06712         0.05526             0.045630   
46           0.08600           0.05943         0.01588             0.005917   
42           0.09081           0.21900         0.21070             0.099610   
180          0.10940           0.19140         0.28710             0.187800   

     ...  texture_worst  perimeter_wor

In [48]:
# We tellen het aantal diagnoses in de training data
diagnosesTotaal = trainingData['diagnosis'].value_counts()

print(diagnosesTotaal)

aantalGoedaardig = diagnosesTotaal[0]
aantalKwaadaardig = diagnosesTotaal[1]

print(f"Er zijn {aantalGoedaardig} goedaardige tumoren en {aantalKwaadaardig} kwaadaardige tumoren in de training data.")

B    176
M    109
Name: diagnosis, dtype: int64
Er zijn 176 goedaardige tumoren en 109 kwaadaardige tumoren in de training data.


In [49]:
# We selecteren de kolommen met de features (x-waarden)
trainingFeatures = trainingData.iloc[:,2:32]
validatieFeatures = validatieData.iloc[:,2:32]
testFeatures = testData.iloc[:,2:32]

# We selecteren de kolom met de diagnoses (y-waarden)
trainingDiagnoses = trainingData.iloc[:,1]
validatieDiagnoses = validatieData.iloc[:,1]
testDiagnoses = testData.iloc[:,1]

In [50]:
print(trainingFeatures)

     radius_mean  texture_mean  perimeter_mean  area_mean  smoothness_mean  \
317       18.220         18.87          118.70     1027.0          0.09746   
508       16.300         15.70          104.70      819.8          0.09427   
46         8.196         16.84           51.71      201.9          0.08600   
42        19.070         24.81          128.30     1104.0          0.09081   
180       27.220         21.87          182.10     2250.0          0.10940   
..           ...           ...             ...        ...              ...   
560       14.050         27.15           91.38      600.4          0.09929   
70        18.940         21.31          123.60     1130.0          0.09009   
559       11.510         23.93           74.52      403.5          0.09261   
110        9.777         16.99           62.50      290.2          0.10370   
0         17.990         10.38          122.80     1001.0          0.11840   

     compactness_mean  concavity_mean  concave points_mean  sym

In [51]:
print(trainingDiagnoses)

317    M
508    B
46     B
42     M
180    M
      ..
560    B
70     M
559    B
110    B
0      M
Name: diagnosis, Length: 285, dtype: object


In [52]:
# we halen de waardes van de feautures op
trainingX = trainingFeatures.values
validatieX = validatieFeatures.values
testX = testFeatures.values

print(trainingX)

[[1.822e+01 1.887e+01 1.187e+02 ... 1.776e-01 2.812e-01 8.198e-02]
 [1.630e+01 1.570e+01 1.047e+02 ... 1.357e-01 2.300e-01 7.230e-02]
 [8.196e+00 1.684e+01 5.171e+01 ... 2.564e-02 3.105e-01 7.409e-02]
 ...
 [1.151e+01 2.393e+01 7.452e+01 ... 9.653e-02 2.112e-01 8.732e-02]
 [9.777e+00 1.699e+01 6.250e+01 ... 5.334e-02 2.533e-01 8.468e-02]
 [1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]]


In [53]:
# we halen de waardes van de diagnoses op
trainingY = trainingDiagnoses.values
validatieY = validatieDiagnoses.values
testY = testDiagnoses.values

print(trainingY)

['M' 'B' 'B' 'M' 'M' 'M' 'B' 'B' 'B' 'B' 'B' 'M' 'B' 'M' 'B' 'B' 'M' 'M'
 'B' 'M' 'B' 'B' 'B' 'M' 'B' 'B' 'M' 'M' 'M' 'B' 'B' 'M' 'M' 'B' 'B' 'B'
 'M' 'M' 'B' 'B' 'B' 'B' 'M' 'M' 'M' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'M' 'M'
 'B' 'M' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'M' 'M' 'B' 'M' 'B'
 'M' 'B' 'B' 'B' 'M' 'B' 'M' 'B' 'B' 'M' 'B' 'M' 'B' 'B' 'B' 'M' 'B' 'M'
 'B' 'M' 'M' 'B' 'B' 'M' 'B' 'M' 'B' 'B' 'M' 'B' 'B' 'M' 'B' 'M' 'B' 'B'
 'B' 'B' 'B' 'B' 'B' 'M' 'M' 'B' 'B' 'M' 'B' 'B' 'B' 'M' 'B' 'M' 'B' 'B'
 'M' 'B' 'B' 'B' 'B' 'B' 'M' 'M' 'M' 'B' 'B' 'M' 'B' 'B' 'M' 'M' 'M' 'B'
 'B' 'M' 'B' 'B' 'B' 'B' 'M' 'B' 'M' 'M' 'B' 'B' 'M' 'M' 'B' 'M' 'B' 'B'
 'M' 'M' 'B' 'B' 'B' 'M' 'B' 'B' 'M' 'B' 'M' 'M' 'M' 'B' 'B' 'B' 'B' 'B'
 'M' 'B' 'M' 'M' 'B' 'B' 'B' 'B' 'M' 'B' 'M' 'M' 'B' 'B' 'M' 'M' 'B' 'M'
 'B' 'B' 'M' 'M' 'B' 'M' 'B' 'B' 'B' 'M' 'M' 'B' 'M' 'B' 'B' 'M' 'B' 'M'
 'M' 'B' 'B' 'M' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'M' 'B' 'B'
 'M' 'B' 'M' 'B' 'B' 'M' 'M' 'B' 'B' 'B' 'M' 'B' 'B

In [54]:
# Dit is de classifier of classificator
# We gebruiken de kernel trick om de data te kunnen scheiden
# In dit geval gebruiken we een lineaire kernel
clf = svm.SVC(kernel='linear')

# We trainen de classifier met de training data
clf.fit(trainingX,trainingY)

# We voorspellen de diagnoses van de validatie data
validatieVoorspellingen = clf.predict(validatieX)

print(validatieVoorspellingen)

['M' 'M' 'B' 'B' 'B' 'B' 'B' 'M' 'B' 'B' 'B' 'B' 'B' 'M' 'M' 'B' 'B' 'B'
 'B' 'B' 'M' 'M' 'B' 'M' 'B' 'M' 'M' 'B' 'M' 'B' 'B' 'B' 'M' 'B' 'M' 'M'
 'M' 'M' 'B' 'B' 'B' 'B' 'M' 'M' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'M'
 'B' 'M' 'M' 'M' 'B' 'B' 'M' 'B' 'B' 'M' 'B' 'B' 'B' 'M' 'B' 'B' 'M' 'B'
 'M' 'B' 'M' 'B' 'B' 'B' 'M' 'B' 'M' 'B' 'B' 'M' 'M' 'B' 'B' 'M' 'B' 'B'
 'B' 'M' 'B' 'M' 'B' 'B' 'B' 'B' 'M' 'B' 'B' 'B' 'M' 'B' 'B' 'B' 'B' 'M'
 'M' 'B' 'M' 'B' 'B' 'B' 'M' 'B' 'M' 'B' 'M' 'B' 'B' 'B' 'M' 'B' 'B' 'B'
 'B' 'B' 'M' 'M' 'B' 'B' 'B' 'B' 'M' 'M' 'M' 'B' 'B' 'B' 'B']


In [55]:
# we berekenen de accuraatheid van de voorspellingen
accuraatheid = metrics.accuracy_score(validatieY, validatieVoorspellingen)
print(f"De accuraatheid van de voorspellingen is {accuraatheid*100:.2f}%")

De accuraatheid van de voorspellingen is 94.33%


### Validatiegegevens gebruiken om de 3 meta-parameters te bepalen.

#### C-term<br>
De C-term staat voor de __foutentoleratie__ van het model.<br><br>
*Kleine waarden voor C:* <br>
Wanneer C klein is, laat het model meer fouten toe, waardoor de beslissingsgrens eenvoudiger wordt.<br><br>
*Grote waarden voor C:* <br>
Wanneer C groot is, laat het model een laag aantal fouten toe, waardoor de beslissingsgrens complexer wordt.<br>

In [56]:
# De C-term is een metaparameter en staat voor de foutentoleratie van het model.
mogelijkeCs = [c for c in range(1, 10)]

hoogsteAccuraatheid = 0
cOpt = 0

# Overloop verschillende waarden voor C
for C in mogelijkeCs:

    clf = svm.SVC(kernel='linear', C=C)
    clf.fit(trainingX, trainingY)
    validatieVoorspellingen = clf.predict(validatieX)
    accuraatheid = metrics.accuracy_score(validatieY, validatieVoorspellingen)

    print(f"De accuraatheid van de voorspellingen op de validatiedata is {accuraatheid*100:.5f}% met C={C}")

    if accuraatheid > hoogsteAccuraatheid:
        hoogsteAccuraatheid = accuraatheid
        cOpt = C

print(f"\nDe optimale waarde voor C is {cOpt} met een accuraatheid van {hoogsteAccuraatheid*100:.2f}%")

De accuraatheid van de voorspellingen op de validatiedata is 94.32624% met C=1
De accuraatheid van de voorspellingen op de validatiedata is 94.32624% met C=2
De accuraatheid van de voorspellingen op de validatiedata is 93.61702% met C=3
De accuraatheid van de voorspellingen op de validatiedata is 92.90780% met C=4
De accuraatheid van de voorspellingen op de validatiedata is 92.19858% met C=5
De accuraatheid van de voorspellingen op de validatiedata is 93.61702% met C=6
De accuraatheid van de voorspellingen op de validatiedata is 92.90780% met C=7
De accuraatheid van de voorspellingen op de validatiedata is 93.61702% met C=8
De accuraatheid van de voorspellingen op de validatiedata is 93.61702% met C=9

De optimale waarde voor C is 1 met een accuraatheid van 94.33%


#### Kernel<br>
De kernel zegt het model welke 'trick' het het moet gebruiken om de data te scheiden.

In [57]:
# De kernel zegt het model welke 'trick' het het moet gebruiken om de data te scheiden.
mogelijkeKernels = ['linear', 'poly', 'rbf', 'sigmoid']

optKernel = ""
hoogsteAccuraatheid = 0

for kernel in mogelijkeKernels:

    clf = svm.SVC(kernel=kernel, C=cOpt)
    clf.fit(trainingX, trainingY)
    validatieVoorspellingen = clf.predict(validatieX)
    accuraatheid = metrics.accuracy_score(validatieY, validatieVoorspellingen)

    print(f"De accuraatheid van de voorspellingen op de validatiedata is {accuraatheid*100:.5f}% met de kernel '{kernel}' en C={cOpt}")

    if accuraatheid > hoogsteAccuraatheid:
        hoogsteAccuraatheid = accuraatheid
        optKernel = kernel

print(f"\nDe optimale kernel is '{optKernel}' met een accuraatheid van {hoogsteAccuraatheid*100:.2f}% voor C={cOpt}")

De accuraatheid van de voorspellingen op de validatiedata is 94.32624% met de kernel 'linear' en C=1
De accuraatheid van de voorspellingen op de validatiedata is 90.78014% met de kernel 'poly' en C=1
De accuraatheid van de voorspellingen op de validatiedata is 90.78014% met de kernel 'rbf' en C=1
De accuraatheid van de voorspellingen op de validatiedata is 49.64539% met de kernel 'sigmoid' en C=1

De optimale kernel is 'linear' met een accuraatheid van 94.33% voor C=1


#### Gamma<br>
De $\gamma$ staat voor de __foutentoleratie__ van het model.<br><br>
*Kleine waarden voor $\gamma$:* <br>
Een te lage gamma-waarde kan leiden tot _overgeneralisatie_, waarbij de beslissingsgrens te eenvoudig is en het model niet goed presteert op de trainingsgegevens.<br><br>
*Grote waarden voor $\gamma$:* <br>
Een te hoge gamma-waarde kan leiden tot _overfitting_, waarbij het model de trainingsgegevens te nauw volgt en dus niet goed zal presteren op nog niet eerder geziene gegevens.<br>

In [58]:
# De C-term is een metaparameter en staat voor de foutentoleratie van het model.
mogelijkeGammas = [gamma for gamma in range(1, 10)]

hoogsteAccuraatheid = 0
gammaOpt = 0

# Overloop verschillende waarden voor C
for gamma in mogelijkeGammas:

    clf = svm.SVC(kernel=optKernel, C=cOpt, gamma=gamma)
    clf.fit(trainingX, trainingY)
    validatieVoorspellingen = clf.predict(validatieX)
    accuraatheid = metrics.accuracy_score(validatieY, validatieVoorspellingen)

    print(f"De accuraatheid van de voorspellingen op de validatiedata is {accuraatheid*100:.5f}% met gamma={gamma}, de kernel '{optKernel}' en C={cOpt}")

    if accuraatheid > hoogsteAccuraatheid:
        hoogsteAccuraatheid = accuraatheid
        gammaOpt = gamma

print(f"\nDe optimale waarde voor gamma is {gammaOpt} met een accuraatheid van {hoogsteAccuraatheid*100:.2f}% voor de kernel '{optKernel}' en C={cOpt}")

De accuraatheid van de voorspellingen op de validatiedata is 94.32624% met gamma=1, de kernel 'linear' en C=1
De accuraatheid van de voorspellingen op de validatiedata is 94.32624% met gamma=2, de kernel 'linear' en C=1
De accuraatheid van de voorspellingen op de validatiedata is 94.32624% met gamma=3, de kernel 'linear' en C=1
De accuraatheid van de voorspellingen op de validatiedata is 94.32624% met gamma=4, de kernel 'linear' en C=1
De accuraatheid van de voorspellingen op de validatiedata is 94.32624% met gamma=5, de kernel 'linear' en C=1
De accuraatheid van de voorspellingen op de validatiedata is 94.32624% met gamma=6, de kernel 'linear' en C=1
De accuraatheid van de voorspellingen op de validatiedata is 94.32624% met gamma=7, de kernel 'linear' en C=1
De accuraatheid van de voorspellingen op de validatiedata is 94.32624% met gamma=8, de kernel 'linear' en C=1
De accuraatheid van de voorspellingen op de validatiedata is 94.32624% met gamma=9, de kernel 'linear' en C=1

De optima

### Toepassen op de testdata

In [59]:
clf = svm.SVC(kernel=optKernel, C=cOpt, gamma=gammaOpt)
clf.fit(trainingX, trainingY)
testVoorspellingen = clf.predict(testX)
accuraatheid = metrics.accuracy_score(testY, testVoorspellingen)

print(f"De accuraatheid van de voorspellingen op de testdata is {accuraatheid*100:.2f}% voor de kernel '{optKernel}', C={cOpt} en gamma={gammaOpt}")

De accuraatheid van de voorspellingen op de testdata is 98.60% voor de kernel 'linear', C=1 en gamma=1
